In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import pandas as pd
import math

In [2]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y1.csv')
data.describe()

,MemberID,ClaimsTruncated,TARGET,trainset,age_05,age_15,age_25,age_35,age_45,age_55,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
count,7.603700e+04,76037.000000,76037.000000,76037.0,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,...,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000,76037.000000
mean,4.994795e+07,0.052225,0.467102,1.0,0.085866,0.090824,0.059405,0.096098,0.132935,0.112366,...,2.175033,0.968765,1.511600,3.703697,3.758749,2.178216,2.885066,1.580296,0.299999,0.344569
std,2.890149e+07,0.222481,1.612204,0.0,0.280168,0.287361,0.236383,0.294727,0.339507,0.315819,...,2.243436,1.046544,1.536634,4.089940,3.434990,2.533432,2.696510,1.814171,0.458260,0.475231
min,2.100000e+02,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.483922e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.990758e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,1.200000,2.000000,3.000000,1.000000,2.600000,1.000000,0.000000,0.000000
75%,7.510115e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.285714,7.000000,6.000000,3.000000,4.800000,2.000000,1.000000,1.000000
max,9.999863e+07,1.000000,15.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,7.000000,7.000000,7.000000,12.000000,10.000000,10.000000,10.000000,12.000000,1.000000,1.000000


In [3]:
data.drop("MemberID", axis = 1, inplace = True)
data.drop("Year", axis = 1, inplace = True)

y = data['TARGET']
X = data.loc[:, data.columns != 'TARGET']
X.head()

,ClaimsTruncated,trainset,age_05,age_15,age_25,age_35,age_45,age_55,age_65,age_75,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
0,0,1,0,0,0,0,0,1,0,0,...,0,0,0.00000,0,10,10,10.000000,1,0,1
1,0,1,0,0,0,0,0,0,0,1,...,5,2,3.90909,11,6,1,2.750000,4,0,0
2,0,1,0,0,0,0,0,0,0,1,...,0,0,0.00000,0,6,6,6.000000,1,0,1
3,0,1,0,0,0,0,0,0,0,1,...,4,1,2.00000,6,6,1,3.333333,3,0,0
4,0,1,0,0,0,0,0,0,0,1,...,2,2,2.00000,1,5,2,2.750000,4,0,0


In [4]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

scaler = StandardScaler().fit(X)
X = scaler.transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

In [6]:
def eval(model, X_test, y_test):
    pred_y = model.predict(X_test) # your predicted y values
    # The root mean square logarithmic error
    pred_y = pred_y.astype(int)
    msle = np.mean( (np.log(pred_y + 1) - np.log(y_test + 1)) ** 2)
    rmsle = math.sqrt(msle)
    print ("RMSLE: %.2f" % rmsle)
    from sklearn.metrics import r2_score
    r2 = r2_score(pred_y, y_test)
    print ("R2 value: %.2f" % r2)

In [7]:
# train with Gradient Boosting algorithm
# compute the accuracy scores on train and validation sets when training with different learning rates

learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = ensemble.GradientBoostingRegressor(n_estimators=20, learning_rate = learning_rate, max_features=4, max_depth = 4, random_state = 0)
    model.fit(X_train, y_train)
    print("Learning rate: ", learning_rate)
    print("Accuracy score (training): {0:.3f}".format(model.score(X_train, y_train)))
    print("Accuracy score (validation): {0:.3f}".format(model.score(X_test, y_test)))
    print()

Learning rate:  0.05
Accuracy score (training): 0.048
Accuracy score (validation): 0.040

Learning rate:  0.1
Accuracy score (training): 0.070
Accuracy score (validation): 0.052

Learning rate:  0.25
Accuracy score (training): 0.104
Accuracy score (validation): 0.061

Learning rate:  0.5
Accuracy score (training): 0.127
Accuracy score (validation): 0.058

Learning rate:  0.75
Accuracy score (training): 0.128
Accuracy score (validation): 0.035

Learning rate:  1
Accuracy score (training): 0.133
Accuracy score (validation): -0.005



In [28]:


# parameters = {
#     "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
#     "min_samples_split": [25,50,60],
#     "min_samples_leaf": np.linspace(0.1, 0.5, 12),
#     "max_depth":[5,8,12],
#     "max_features":["log2","sqrt"],
#     "n_estimators":[100, 200, 500]
#     }

# model = GridSearchCV(ensemble.GradientBoostingRegressor(learning_rate=0.005,n_estimators=500,max_depth=4,min_samples_split=50, verbose =1), parameters, cv=10, n_jobs=-1)
from sklearn import linear_model

    
model= ensemble.GradientBoostingRegressor(
                                          n_estimators=1000,
                                          learning_rate= 0.002,
                                          max_depth = 7,
                                          min_samples_leaf=100,
                                          verbose =1)
model.fit(X_train, y_train)
eval(model, X_test, y_test)

      Iter       Train Loss   Remaining Time 
         1           2.6459           20.20m
         2           2.6449           20.48m
         3           2.6439           20.28m
         4           2.6429           20.15m
         5           2.6418           20.08m
         6           2.6408           20.05m
         7           2.6398           20.40m
         8           2.6388           20.48m
         9           2.6379           20.40m
        10           2.6369           20.46m
        20           2.6272           22.46m
        30           2.6178           22.80m
        40           2.6087           22.20m
        50           2.5999           21.69m
        60           2.5914           21.24m
        70           2.5831           20.85m
        80           2.5752           20.43m
        90           2.5674           20.20m
       100           2.5600           19.98m
       200           2.4966           17.58m
       300           2.4473           15.57m
       40

In [8]:
data_submit = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
data_submit.describe()

,MemberID,ClaimsTruncated,TARGET,trainset,age_05,age_15,age_25,age_35,age_45,age_55,...,drugCount_max,drugCount_min,drugCount_ave,drugcount_months,labCount_max,labCount_min,labCount_ave,labcount_months,labNull,drugNull
count,7.143500e+04,71435.000000,71435.000000,71435.0,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,...,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000,71435.000000
mean,4.993986e+07,0.056611,0.437895,1.0,0.088108,0.085364,0.057213,0.096563,0.136474,0.116232,...,2.207475,0.948695,1.516412,3.864030,3.879919,2.180024,2.937653,1.713670,0.282313,0.353090
std,2.894077e+07,0.231099,1.531780,0.0,0.283454,0.279425,0.232251,0.295364,0.343293,0.320505,...,2.281410,1.040350,1.559578,4.210155,3.421071,2.471471,2.640371,1.975234,0.450128,0.477934
min,4.000000e+00,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.477813e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.995379e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,1.000000,1.222222,2.000000,4.000000,1.000000,3.000000,1.000000,0.000000,0.000000
75%,7.521986e+07,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.333333,8.000000,7.000000,3.000000,4.800000,2.000000,1.000000,1.000000
max,9.999882e+07,1.000000,15.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,7.000000,7.000000,7.000000,12.000000,10.000000,10.000000,10.000000,12.000000,1.000000,1.000000


In [9]:
data_submit.drop("MemberID", axis = 1, inplace = True)
data_submit.drop("Year", axis = 1, inplace = True)

y_submit = data_submit['TARGET']
X_submit = data_submit.loc[:, data_submit.columns != 'TARGET']
y_submit.describe()

count    71435.000000
mean         0.437895
std          1.531780
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         15.000000
Name: TARGET, dtype: float64

In [10]:
X_submit = scaler.transform(X_submit)

1st time: 0.49

In [29]:
eval(model, X_submit, y_submit)

RMSLE: 0.48
R2 value: -17.23


In [12]:
data = pd.read_csv('../1.data-preparation/new-data/data_Y2.csv')
pred_y = model.predict(X_submit)

submission = pd.DataFrame()
submission['MemberID'] = data['MemberID']
submission['DIH'] = pred_y.astype(int)
submission.head()
submission.groupby(['DIH']).size()

DIH
0    67182
1     4135
2      118
dtype: int64

In [24]:
submission.groupby(['DIH']).size()

DIH
0     71048
1        93
2        75
3       126
4        30
5        12
6         2
7         3
8         8
9         4
12        7
13        1
14        7
15       19
dtype: int64